<a href="https://colab.research.google.com/github/skyprince999/100-Days-Of-ML/blob/master/Day%2026%23%20Analyzing_Tweets_PySpark_Batch_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Install spark dependencies

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.6.tgz

In [ ]:
!tar xf spark-2.4.6-bin-hadoop2.6.tgz

In [ ]:
!pip install -q findspark
!pip install pyspark

In [ ]:
!wget http://apachemirror.wuchna.com/hadoop/common/stable/hadoop-3.2.1.tar.gz
!tar xf hadoop-3.2.1.tar.gz

### Setup required environment variables

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.6"
os.environ['HADOOP_HOME'] = '/content/hadoop-3.2.1'

### Import required libraries

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql.functions import isnan, when, count, col

In [ ]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

import json

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Connect to the Spark server
SparkContext.setSystemProperty('spark.executor.memory', '4g')

spark = SparkSession.builder.appName("Covid-Tweet").master("local[*]").getOrCreate()


In [ ]:
%%time
!mkdir SPARK-2020-01
!cp drive/"My Drive"/"SPARK+AI Hackathon 2020"/SPARK-2020-05/*.*  SPARK-2020-01/

!cp drive/"My Drive"/"SPARK+AI Hackathon 2020"/SPARK-2020-01/*.*  SPARK-2020-01/

CPU times: user 1.16 s, sys: 202 ms, total: 1.36 s
Wall time: 8min 11s


In [ ]:
# filename05 = os.listdir('SPARK-2020-05')
# filename05 = [ 'SPARK-2020-05/' + f for f in filename05]
# print(len(filename05))

# filename01 = os.listdir('SPARK-2020-01')
# filename01 = [ 'SPARK-2020-01/' + f for f in filename01]
# print(len(filename01))

In [ ]:
%%time
# Create data frame. spark.read.json >>> can also read from a directory 
json_file_path = 'SPARK-2020-01'
df = spark.read.json(json_file_path)

CPU times: user 234 ms, sys: 59.6 ms, total: 294 ms
Wall time: 25min 8s


In [ ]:
# The inferred schema can be visualized using the printSchema() method
#df.printSchema()

#df.show(5)

#df.dtypes

#print((df.count(), len(df.columns)))

# Creates a temporary view using the DataFrame
#df.createOrReplaceTempView("tweets")

# SQL statements can be run by using the sql methods provided by spark
#tweetsDF = spark.sql("SELECT * FROM tweets")
#tweetsDF.show(10)

#df.describe('favorite_count').show()

# The following code block gives a null value

# from pyspark.sql import functions as F

# df2 = spark.createDataFrame([('Tue May 05 16:14:52 +0000 2020',)], ['t'])
# df2 = df2.withColumn('new_date', df2.t.substr(1,19))
# df2 = df2.withColumn('dt', F.to_date(df2.new_date, 'EEE MMM dd HH:MM:SS'))
# print(df2.show())

# This gives a OOM error 
# result = df.select('*').toPandas()

## Instead add limit(n)
# result = df.select('*').limit(100).toPandas()

# df = df.withColumn('Day', df.created_at.substr(9,10))
# df = df.withColumn('Hour', df.created_at.substr(12,13))

# df.show(10, truncate=False)

# df.select('entities', 'full_text').show(100,truncate=False)

# from pyspark.sql.functions import flatten
# df.select(df.full_text,flatten(df.entities)).show(100, truncate=False)

# df.select("full_text" , "user").show(10, truncate=False)

# # Creates a temporary view using the DataFrame
# df.createOrReplaceTempView("tweets")

# # SQL statements can be run by using the sql methods provided by spark
# tweetsDF = spark.sql("SELECT * FROM tweets").limit(100)
# #tweetsDF.show(10)

In [ ]:
df.show(5)

+------------+-----------+--------------------+------------------+--------------------+-----------------+--------------+---------+--------------------+----+-------------------+-------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+---------------+----+-----+------------------+-------------+----------------+--------------------+-----------------------+-------------+---------+--------------------+--------------------+---------+--------------------+------------------+---------------------+--------------+
|contributors|coordinates|          created_at|display_text_range|            entities|extended_entities|favorite_count|favorited|           full_text| geo|                 id|             id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|is_quote_status|lang|place|possibly_sensitive|quoted_status|quoted_status_id|quoted_status_id_str|qu

In [ ]:
df.count()

17165069

In [ ]:
def user(text):
  return text['screen_name']

def hashtag(text):
  return [h[1] for h in text[0]]
  #return text[0]

def user_mention(text):
  return [h[-1] for h in text[-1]]

In [ ]:
from pyspark.sql.types import ArrayType 

# Define your function
getSN = UserDefinedFunction(lambda x: user(x), StringType())
getHash = UserDefinedFunction(lambda x: hashtag(x), ArrayType(StringType()))
getUM = UserDefinedFunction(lambda x: user_mention(x), ArrayType(StringType()))

In [ ]:
user = df.withColumn('user', getSN(col('user'))) #.show(5)  #<<- This works 

In [ ]:
user = user.withColumn('entities', explode(array('entities'))) #.show(5)

In [ ]:
user = user.withColumn('hashtags', getHash('entities')) #.show(50)

In [ ]:
user = user.withColumn('user_mention', getUM('entities'))

In [ ]:
user.select('user', 'hashtags', 'user_mention', 'full_text').show(50)

+---------------+--------------------+--------------------+--------------------+
|           user|            hashtags|        user_mention|           full_text|
+---------------+--------------------+--------------------+--------------------+
|   ivaforgotton|                  []|                  []|Coronavirus inter...|
|     iivanov482|[coronavirus, Cor...|       [evankirstel]|RT @evankirstel: ...|
|websterelizabet|                  []|       [CapriLaptop]|RT @CapriLaptop: ...|
|newagebitchface|                  []|   [ElliottKaybooks]|RT @ElliottKayboo...|
|     rtsang0718|                  []|         [PostWorld]|RT @PostWorld: In...|
|        mvintel|                  []|       [ali20waheed]|@ali20waheed stil...|
|     ramzyyyyy_|                  []|                  []|CNN: McGill Unive...|
|        linod99|                  []|         [emtpyvase]|RT @emtpyvase: so...|
|   Iamathelstan|                  []|           [FLUMEED]|RT @FLUMEED: Aucu...|
|   MableKipenda|           

In [ ]:
query = user.select('user', 'hashtags', 'user_mention', 'full_text')
query.dtypes # csv can't save a csv file with columns in array<strin> format

[('user', 'string'),
 ('hashtags', 'array<string>'),
 ('user_mention', 'array<string>'),
 ('full_text', 'string')]

In [ ]:
sparse_format_udf = udf(lambda x: ','.join([str(elem) for elem in x], StringType()))

query = query.withColumn('hashtags', sparse_format_udf(col('hashtags')))
query = query.withColumn('user_mention', sparse_format_udf(col('user_mention')))
query.dtypes

[('user', 'string'),
 ('hashtags', 'string'),
 ('user_mention', 'string'),
 ('full_text', 'string')]

In [ ]:
type(query)

pyspark.sql.dataframe.DataFrame

In [ ]:
'''
The following statements are causing an error 
>>>
raise Py4JError(

An error occurred while calling o278.collectToPython

TypeError: join() takes exactly one argument (2 given)
'''
# query.limit(1000).toPandas().to_excel('Output.xlsx')

# # Save file local folder, delimiter by default is ,
# query.coalesce(1).write.format('csv').option('header',"True").mode('overwrite').option('sep',',').save('output')

# query.write.format("parquet").save("jan2020.parquet")

'\nThe following statements are causing an error \n>>>\nraise Py4JError(\n\nAn error occurred while calling o278.collectToPython\n\nTypeError: join() takes exactly one argument (2 given)\n'